# Conver generated caption S-BERT file format

* IMLAB으로부터 전달받은 S-BERT 파일을 우리가 원하는 형식으로 변환한다.

In [27]:
import numpy as np
import h5py
import pickle
from tqdm import tqdm

In [3]:
ori_npy = np.load('/data/project/rw/CBIR/data/vg_coco/vg_coco_sbert_mean.npy')
ori_idx = np.load('/data/project/rw/CBIR/data/vg_coco/vg_coco_sbert_img_id.npy')

In [4]:
ori_npy.shape

(48220, 48220)

In [5]:
ori_idx.shape

(48220,)

In [6]:
gen = h5py.File('/data/public/rw/datasets/visual_genome/BERT_feature/SBERT_gen_sims_new.hdf5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """Entry point for launching an IPython kernel.


In [7]:
gen_id = gen['id']

In [8]:
len(gen_id)

51498

In [9]:
'''id가 전부 inclusive한 것이 맞음'''
set(ori_idx).difference(set(gen_id))

set()

In [10]:
new_npy_path = '/data/project/rw/CBIR/data/vg_coco/vg_coco_gencap_sbert.npy'
new_idx_path = '/data/project/rw/CBIR/data/vg_coco/vg_coco_gencap_sbert_img_id.npy'

In [11]:
np.save(new_idx_path, ori_idx)  # 걍 복사해도 됨. id는 같아야하니까

In [13]:
gen_id[0]

2415074

In [17]:
'''최대 최소가 2, 0 으로 맞추어져 있는 것 확인. 보정해야함.'''
sim_arr = gen['sims'][str(gen_id[0])][:]
print(sim_arr.max(), sim_arr.min())

In [21]:
set_ori_idx = set(ori_idx)
mask = [True if id_ in set_ori_idx else False for id_ in gen['id']]

In [24]:
ori_npy.max(), ori_npy.min()

(0.9843528, -0.31441414)

In [28]:
A = np.zeros_like(ori_npy)
for i, id_ in enumerate(tqdm(ori_idx)):
    A[i] = gen['sims'][str(id_)][:][mask] - 1

100%|██████████| 48220/48220 [03:53<00:00, 206.34it/s]


In [29]:
A.shape

(48220, 48220)

In [31]:
np.save(new_npy_path, A)

In [32]:
a = pickle.load(open('/data/project/rw/CBIR/data/vg_coco/vg_coco_split.pkl', 'rb'))

In [36]:
a['train'][0] in set_ori_idx

True